## DETECTOR

In [4]:
import cv2


faceCascade = cv2.CascadeClassifier('Cascades/haarcascade_frontalface_default.xml')
cap = cv2.VideoCapture(0)

face_id = input('enter user id ')
print('[INFO] Initialising face recoginition. Look at the camera for 120 seconds...')

count=0
while True:
    success, img = cap.read()
    img = cv2.flip(img, 1)
    #drawing face detection annotations
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(
        gray,     
        scaleFactor=1.2,
        minNeighbors=5,     
    )
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        count+=1

        #save sample images for training data
        cv2.imwrite("dataset/User."+str(face_id)+'.'+str(count)+".jpg", gray[y:y+h, x:x+w])
        cv2.imshow('Image', img)
        
    k= cv2.waitKey(100) & 0xff
    if k==27:
        break
    elif count>=120:
        break
print('[EXITING] Exiting program and cleanup stuff')
cap.release()
cv2.destroyAllWindows()

enter user id 001
[INFO] Initialising face recoginition. Look at the camera for 120 seconds...
[EXITING] Exiting program and cleanup stuff


## TRAINER

In [ ]:
!pip install --user opencv-contrib-python

In [5]:
import cv2
import numpy as np
from PIL import Image
import os

path = 'dataset'
recognizer = cv2.face.LBPHFaceRecognizer_create()
detector = cv2.CascadeClassifier("Cascades/haarcascade_frontalface_default.xml")

#get images and face data
def getImagesAndLabels(path):
    imagePaths = [os.path.join(path, f) for f in os.listdir(path)]
    faceSamples = []
    ids = []
    for imagePath in imagePaths:
        PIL_img = Image.open(imagePath).convert('L')  #convert to grayscale
        img_numpy = np.array(PIL_img, 'uint8')
        id = int(os.path.split(imagePath)[-1].split(".")[1])
        faces = detector.detectMultiScale(img_numpy)
        for (x,y,w,h) in faces:
            faceSamples.append(img_numpy[y:y+h, x:x+w])
            ids.append(id)
    return faceSamples, ids


print('\n[INFO] Training faces... Please Wait...')

faces, ids = getImagesAndLabels(path)
recognizer.train(faces, np.array(ids))

#save the model to trainer.yml
recognizer.write('trainer/trainer.yml')

#print number of faces trained
print('\n [INFO] {0} faces trained. Exiting program..'.format(len(np.unique(ids))))


[INFO] Training faces... Please Wait...

 [INFO] 1 faces trained. Exiting program..


## RECOGNITION 

In [6]:
import cv2
import numpy as np
import os

recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('trainer/trainer.yml')
cascadePath = "Cascades/haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascadePath)
font = cv2.FONT_HERSHEY_SIMPLEX

#id counter 
id=0
names = ['NONE','ANUBHAV']

cam = cv2.VideoCapture(0)

count=0

In [ ]:
while True:
    ret, img = cam.read()
    img = cv2.flip(img,1)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = faceCascade.detectMultiScale(gray, scaleFactor = 1.2, minNeighbors = 5)

    for(x,y,w,h) in faces:
        cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)
        id, confidence = recognizer.predict(gray[y:y+h,x:x+w])
        
        # If confidence is less them 100 ==> "0" : perfect match 
        if (confidence > 50):
            id = names[id]
            confidence = "  {0}%".format(round(100 - confidence))
        else:
            id = "unknown"
            confidence = "  {0}%".format(round(100 - confidence))
        
        cv2.putText(
                    img, 
                    str(id), 
                    (x+5,y-5), 
                    font, 
                    1, 
                    (255,255,255), 
                    2
                   )
        cv2.putText(
                    img, 
                    str(confidence), 
                    (x+5,y+h-5), 
                    font, 
                    1, 
                    (255,255,0), 
                    1
                   ) 
        cv2.imshow('camera',img) 
        k = cv2.waitKey(10) & 0xff # Press 'ESC' for exiting video
        if k == 40:
            break

In [ ]:
# Do a bit of cleanup
print("\n [INFO] Exiting Program and cleanup stuff")
cam.release()
cv2.destroyAllWindows()